За да стартирате следните тетрадки, ако все още не сте го направили, трябва да зададете openai ключа във файла .env като `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

След това ще заредим Индекса на вгражданията в Pandas Dataframe. Индексът на вгражданията е съхранен в JSON файл, наречен `embedding_index_3m.json`. Индексът на вгражданията съдържа вгражданията за всеки от YouTube транскриптите до края на октомври 2023 г.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Следващата стъпка е да създадем функция, наречена `get_videos`, която ще търси в Embedding Index за дадената заявка. Функцията ще връща топ 5 видеа, които са най-сходни със заявката. Функцията работи по следния начин:

1. Първо се създава копие на Embedding Index.
2. След това се изчислява Embedding за заявката чрез OpenAI Embedding API.
3. После се създава нова колона в Embedding Index, наречена `similarity`. Тази колона съдържа косинусната прилика между Embedding на заявката и Embedding на всеки видео сегмент.
4. След това Embedding Index се филтрира по колоната `similarity`. Остават само видеата, които имат косинусна прилика по-голяма или равна на 0.75.
5. Накрая Embedding Index се сортира по колоната `similarity` и се връщат топ 5 видеа.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Тази функция е много проста, тя просто отпечатва резултатите от заявката за търсене.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Първо, Embedding Index се зарежда в Pandas Dataframe.
2. След това на потребителя се показва поле за въвеждане на заявка.
3. После се извиква функцията `get_videos`, за да се търси в Embedding Index по въведената заявка.
4. Накрая се извиква функцията `display_results`, която показва резултатите на потребителя.
5. След това потребителят може да въведе нова заявка. Този процес се повтаря, докато потребителят не въведе `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.bg.png)

Ще ви бъде показано поле за въвеждане на заявка. Въведете заявка и натиснете Enter. Приложението ще върне списък с видеа, които са свързани с вашата заявка. Освен това ще получите линк към мястото във видеото, където се намира отговорът на въпроса.

Ето няколко примерни заявки, които можете да пробвате:

- Какво е Azure Machine Learning?
- Как работят конволюционните невронни мрежи?
- Какво е невронна мрежа?
- Мога ли да използвам Jupyter Notebooks с Azure Machine Learning?
- Какво е ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни тълкувания, произтичащи от използването на този превод.
